# Exploring and Visualizing Cultural Bias in Top Ranked Novels

## Introduction
This notebook performs an exploratory data analysis (EDA) on two datasets: **Top 500 Greatest Novels** and **New York Times Hardcover Fiction Bestsellers**. The goal is to uncover patterns, gaps, and biases, focusing on author demographics, genre representation, and historical popularity.


In [1]:
import pandas as pd
import altair as alt

# Load the datasets
top500_df = pd.read_csv("https://raw.githubusercontent.com/melaniewalsh/responsible-datasets-in-context/main/datasets/top-500-novels/library_top_500.csv")
nyt_df = pd.read_csv("https://raw.githubusercontent.com/ecds/post45-datasets/main/nyt_full.tsv", sep="\t")

## Initial Data Overview
We start by inspecting the structure and basic information about each dataset, including missing values and data types.


In [2]:
# Display basic info about the datasets
print("Top 500 Novels Dataset Info:")
top500_df.info()
print("\nNYT Bestsellers Dataset Info:")
nyt_df.info()

Top 500 Novels Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   top_500_rank              500 non-null    int64  
 1   title                     500 non-null    object 
 2   author                    500 non-null    object 
 3   pub_year                  500 non-null    int64  
 4   orig_lang                 500 non-null    object 
 5   genre                     500 non-null    object 
 6   author_birth              499 non-null    object 
 7   author_death              496 non-null    object 
 8   author_gender             499 non-null    object 
 9   author_primary_lang       499 non-null    object 
 10  author_nationality        499 non-null    object 
 11  author_field_of_activity  329 non-null    object 
 12  author_occupation         458 non-null    object 
 13  oclc_holdings             495 non-nu

### Handling Missing Data
We will explore the missing data in the Top 500 Novels dataset and how it may affect our analysis. We use the `isna()` method to inspect missing values in specific columns and visualize their distribution.


In [3]:
# Check for missing values in key columns
missing_data = top500_df.isna().sum()
print("\nMissing Data Overview:")
print(missing_data)


Missing Data Overview:
top_500_rank                  0
title                         0
author                        0
pub_year                      0
orig_lang                     0
genre                         0
author_birth                  1
author_death                  4
author_gender                 1
author_primary_lang           1
author_nationality            1
author_field_of_activity    171
author_occupation            42
oclc_holdings                 5
oclc_eholdings                5
oclc_total_editions           5
oclc_holdings_rank            5
oclc_editions_rank            5
gr_avg_rating                 0
gr_num_ratings                0
gr_num_reviews                0
gr_avg_rating_rank            0
gr_num_ratings_rank           0
oclc_owi                      5
author_viaf                   0
gr_url                       20
wiki_url                      0
pg_eng_url                    0
pg_orig_url                 436
dtype: int64


### Merging the Datasets
We want to see if there are any common titles or authors between the datasets to understand the overlap. For this, we will use `merge()` and check the shared information between the two datasets.


In [4]:
# Capitalize the title column for consistency
nyt_df['title'] = nyt_df['title'].str.capitalize()

# Merge the datasets based on common 'title' and 'author'
merged_df = top500_df.merge(nyt_df, how='inner', on=['title', 'author'])
print(f"Number of shared titles: {len(merged_df)}")

Number of shared titles: 231


## Exploratory Data Analysis (EDA)
The next step is to perform an EDA to identify any noticeable patterns or biases. We start by understanding the genre distribution and author demographics in the Top 500 dataset.


In [5]:
# Ensuring the genre column is of string type before plotting
top500_df['genre'] = top500_df['genre'].astype(str)

# Plot the distribution of genres
genre_counts = top500_df['genre'].value_counts().reset_index()
genre_counts.columns = ['Genre', 'Count']

genre_chart = alt.Chart(genre_counts).mark_bar().encode(
    x='Count:Q',
    y=alt.Y('Genre:N', sort='-x'),
    color='Genre:N'
).properties(
    title="Genre Distribution in Top 500 Novels"
)
genre_chart

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### Author Demographics
The chart below shows the distribution of author genders in the dataset, helping us understand whether there are any biases in the gender representation of top-ranked novels.


In [6]:
# Ensuring author_gender is a string type before plotting
top500_df['author_gender'] = top500_df['author_gender'].astype(str)

# Count author genders
gender_counts = top500_df['author_gender'].value_counts().reset_index()
gender_counts.columns = ['Gender', 'Count']

gender_chart = alt.Chart(gender_counts).mark_bar().encode(
    x='Count:Q',
    y=alt.Y('Gender:N', sort='-x'),
    color='Gender:N'
).properties(
    title="Author Gender Distribution in Top 500 Novels"
)
gender_chart

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### Genre Trends Over Time
We analyze how genre popularity has evolved over the years by creating a line chart of genre frequency over time.


In [7]:
# Create a datetime column for the publication year
top500_df['pub_date'] = pd.to_datetime(top500_df['pub_year'].astype(str) + '-01-01', format='%Y-%m-%d', errors='coerce')

# Group and count genres by publication date
genre_over_time = top500_df.groupby(['pub_date', 'genre']).size().reset_index(name='Count')

trend_chart = alt.Chart(genre_over_time).mark_line().encode(
    x='pub_date:T',
    y='Count:Q',
    color='genre:N'
).properties(
    title="Genre Trends Over Time in Top 500 Novels"
)
trend_chart

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## Conclusion
Through our analysis, we observe potential biases in author gender and genre popularity. The merged dataset helps us understand overlaps between the Top 500 Greatest Novels and NYT Bestsellers, showing a trend in shared authors but not necessarily shared titles. This analysis informs future research in literature, helping to understand which genres or authors are given more visibility and why.